# Ensembles de árboles en un dataset de detección de spam

Si bien en anteriores prácticas hemos insistido en emplear los datasets de digits y faces, es bien conocido que para este tipo de datasets que representan imágenes, no es lo habitual emplear árboles de decisión.

Árboles de decisión y todos los ensembles propuestos funcionan en la práctica muy bien en datos tabulares como puede ser el problema de reconocimiento de spam en mails.

Se propone pues emplear el dataset id=44 de openml de detección de Spam. Son un total de 4601 muestras con 57 características.


## Un primer clasificador con un único árbol de decisión

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

## Descarga del dataset Spam
X, y = fetch_openml(data_id=44, as_frame=False, cache=True, return_X_y=True)
print(X.shape)

## Partición train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


## Grid Search
G = {"max_depth": [2,4,6,8,10], "min_samples_split": [2,3,4]}

GS = GridSearchCV(DecisionTreeClassifier(random_state=23), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

(4601, 57)
Precisión: 93.9% con {'max_depth': 10, 'min_samples_split': 4}


## Bagging

Bagging es una técnica genérica de muestreo con remplanzamiento del conjunto de muestras de entrenamiento. Se puede emplear junto con cualquier clasificador para obtener un ensemble. Los parámetros más importantes son por un lado el estimador que se quiere emplear, en nuestro caso un DecisionTree, y el número de estimadores que se quiere combinar en el ensemble.

In [2]:
from sklearn.ensemble import BaggingClassifier

clf= BaggingClassifier(estimator=DecisionTreeClassifier(random_state=23,max_depth=10,min_samples_split=4),n_estimators=10)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')


Precisión: 94.2%


**Ejercicio:** Encuentra los mejores parámetros mediante grid search. En concreto, n_estimators y max_depth de los árboles de decisión empleados. Se debería poder llegar alrededor de un **94.5% de acierto**.  

In [3]:
## Solución

G = {"n_estimators":[1,10,25,50,100],"estimator__max_depth": [2,4,6,8,10]}

GS = GridSearchCV(BaggingClassifier(estimator=DecisionTreeClassifier(random_state=23)), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Precisión: 94.7% con {'estimator__max_depth': 10, 'n_estimators': 50}


## RandomForest

RandomForest es otra técnica de ensemble, en este caso no genérica sino particular para árboles de decisión. Cada árbol se construye empleando una selección aleatoria de las características en cada uno de los nodos. El parámetro más importante es "n_estimators", el número de árboles que combino. El parámetro binario "bootstrap" controla también si se realiza bootstrap de las muestras (Bagging) o no.



In [4]:
from sklearn.ensemble import RandomForestClassifier

clf= RandomForestClassifier(n_estimators=10, max_depth=10, min_samples_split=4, bootstrap=False, random_state=23)

acc = clf.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%}')



Precisión: 94.1%


**Ejercicio:** Realiza una búsqueda de parámetros mediande grid search. En concreto para los parámetros: n_estimators, max_depth y bootstrap. Se debería poder llegar alrededor de un **95% de acierto**.  

In [5]:
## Solución

from sklearn.ensemble import RandomForestClassifier

## Grid Search
G = {"n_estimators":[1,10,25,50,100],"max_depth": [2,4,6,8,10], "bootstrap": [True, False]}

GS = GridSearchCV(RandomForestClassifier(random_state=23), G, scoring='accuracy', refit=True, cv=5)

acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')


Precisión: 95.0% con {'bootstrap': False, 'max_depth': 10, 'n_estimators': 25}


## Ejercicio Final: NO SE VE MEJORA... QUITAR

Prueba además a añadir técnicas de preproceso a las características como puede ser PCA o expansión polinómica para intenetar obtener el mejor resultado posible. Emplea grid search como método de búsqueda de parámetros.


In [6]:
## Solución (unos 5 minutos)

## NO SE CONSIGUE MEJORAR, MEJOR QUITAR ESTE EJERCICIO

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures


poly=PolynomialFeatures()
#pca=PCA()  Con PCA salen peores resultados
rf=RandomForestClassifier(random_state=23)

pipe=Pipeline(steps=[("poly", poly), ("rf", rf)])

G = {"poly__degree": [1,2], "rf__max_depth": [6,8,10,12], "rf__n_estimators": [25,50,100,200,300], "rf__bootstrap": [True, False]}

# activo el verbose para ver lo que tarda cada fit
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5, verbose=10)
acc = GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Precisión: {acc:.1%} con {GS.best_params_}')

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5; 1/80] START poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25
[CV 1/5; 1/80] END poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25;, score=0.932 total time=   0.1s
[CV 2/5; 1/80] START poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25
[CV 2/5; 1/80] END poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25;, score=0.918 total time=   0.1s
[CV 3/5; 1/80] START poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25
[CV 3/5; 1/80] END poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25;, score=0.935 total time=   0.1s
[CV 4/5; 1/80] START poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25
[CV 4/5; 1/80] END poly__degree=1, rf__bootstrap=True, rf__max_depth=6, rf__n_estimators=25;, score=0.923 total time=   0.1s
[CV 5/5; 1/80] START poly__degree=1, rf__bootstrap=True, rf__m